# RNN language modeling and generation

Building a RNN language model from IMDb data comments. The notebook walks you through the construction of a language model based on a LSTM recurrent neural network, following the usual steps: data preparation, vocabulary selection, model definition and training. Once trained, you're invited to use the language model as a text generator: this part is not implemented (yet) but you're provided with a toy generation loop with LSTM networks that should easily be adapated. Note that training LSTMs might take some time. 

## Loading and preparing the data

In [ ]:
#
# load a bunch of modules
#

import json
import numpy as np
import random
from tqdm import tqdm
from collections import Counter
import statistics
import math

from nltk import word_tokenize, sent_tokenize
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
#
# load IMDb data and process a small number of samples (positive here)
#

fn = '../TP/data/imdb-trn.json'

with open(fn, 'rt') as f:
    imdb_data = json.load(f)
    

data = imdb_data[:2000] + imdb_data[-2000:]

In [ ]:
#
# basic data cleansing and preparation to yield two list of utterances: 
# texts[0] and texts[1] contain sentences from positive and negative comments respectively.
#

def clean_utterance(buf):
    '''
    Clean the list of tokens.
    '''
    ignore = ("``", "''", "(", ")", '<', 'br', '/', '>', '--', '*', '-')
    
    return [x.lower() for x in buf if x.lower() not in ignore]


texts = [[], []]

for post in [x[1] for x in data if x[0] == 'pos']:
    for utterance in sent_tokenize(post):
        texts[0].append(clean_utterance(word_tokenize(utterance)))

for post in [x[1] for x in data if x[0] == 'neg']:
    for utterance in sent_tokenize(post):
        texts[1].append(clean_utterance(word_tokenize(utterance)))

print(len(texts[0]), texts[0][0])
print(len(texts[1]), texts[1][0])

## Define vocabulary

In [ ]:
#
# The ususal steps to define the vocabulary, here limiting ourselves to tokens that appear 
# at least 5 times in the positive utterances (we will build a model for positive comments only).
#

counter = Counter()
   
for item in texts[0]:
    counter.update(item)

counter = dict(sorted(counter.items(), key=lambda x: x[1], reverse = True))

#
# Pretty print a number of things
#
print('total number of tokens in dataset =', len(counter))
# print('\nleast frequent tokens:')
# for x in list(counter.keys())[-20:]:
#    print(f"   {x:18}  {counter[x]}")

    
MINOCC = 5 # keep only tokens that appear at least MINOCC times

#
# Special tokens in the vocabulary
#    PAD : padding sequences to the same length
#    BOS : begining of sentence
#    EOS : end of sentence
#    UNK : out-of-vocabulary (OOV) token
#
vocab = {'PAD': 0, 'BOS': 1, 'EOS': 2, 'UNK': 3}
offset = len(vocab)
vocab.update({x: i+offset for i, x in enumerate(list(counter.keys())) if counter[x] >= MINOCC})

print(f'Number of tokens appearing more than {MINOCC} times =', len(vocab) - 4)
print(list(vocab.keys())[:20])

# build inverse mapping
id2str = list(vocab.keys())


## Encode dataset, split into folds and convert to pytorch Dataset

In [ ]:
#
# Encode all utterances given the vocabulary mapping
#

def encode_utterance(x: list[str]) -> list[int]:
    '''
    Encode an utterance according to the mapping provided by vocab. 
    '''
    global vocab
    
    unk_id = vocab['UNK']
    
    return [vocab['BOS']] + [vocab.get(token, unk_id) for token in x]


def decode_utterance(_ids: list[int]) -> str:
    '''
    Returns the string corresponding to the list of ids
    '''
    
    global id2str
    
    return ' '.join([id2str[x] for x in _ids])


#
# Apply encode_utterance() on all positive texts
#
encodings = list(map(encode_utterance, texts[0]))

print(encodings[:10])
print(decode_utterance(encodings[0]))

In [ ]:
#
# Basic statistics on the entire dataset before we split
#

ntokens = [len(x) for x in encodings]

m = statistics.mean(ntokens)
m2 = statistics.median(ntokens)
sdev = statistics.stdev(ntokens)

print('{:35s} min={}  max={}  mean={:.1f}  median={}  sdev={:.1f}'.format('Number of tokens per utterance', min(ntokens), max(ntokens), m, m2, sdev))

n_no_unk = len([x for x in encodings if vocab['UNK'] not in x])
print('Number of utterances with no OOV   ', n_no_unk, 'out of {}'.format(len(encodings)))

In [ ]:
#
# Make splits
#

fold = dict()

fold['train'], buf = train_test_split(encodings, test_size=0.3, random_state=42)
fold['valid'], fold['test'] = train_test_split(buf, test_size=0.5, random_state=42)

print(len(fold['train']), len(fold['valid']), len(fold['test']))

In [ ]:
#
# Create label sequences for a batch of token sequences
#
def make_labels(_data: list[list[int]]) -> list[list[int]]:
    '''
    Shift left input sequence and complete with the EOS token.
    '''
    
    global vocab
    
    eos_id = vocab['EOS']

    return [x[1:] + [eos_id] for x in _data]

#
# Pads a batch of sequences
#
def pad_sequences(_data: list[list[int]], maxlen: int = 0) -> list[list[int]]:
    '''
    Pad all input utterances up to maxlen, truncating if need be. If maxlen is null, will
    be set to the length of the longest sequence
    
    Returns a list with padded ids. 
    '''
    
    global vocab
    
    pad_id = vocab['PAD']

    if maxlen <= 0:
        maxlen = max((len(x) for x in _data))
    
    return [(x + [pad_id] * (maxlen - len(x)))[:maxlen] for x in _data]

In [ ]:
#
# Sanity check of the functions in the previous cell
#

inputs = fold['train'][:2]
print('inputs =', inputs, '/{}/'.format(len(inputs)))
labels = make_labels(inputs)
print('\nlabels = ', labels, '/{}/'.format(len(labels))) 

MAXLEN = 50

print('\n')
padded_inputs = pad_sequences(inputs, MAXLEN)
print(padded_inputs)
print('\n')
print(make_labels(padded_inputs)) 


In [ ]:
#
# Define the dataset class to hold the (padded) input id sequences referred to as 'encodings'
# and the corresponding (padded) label sequences. Note that making labels and padding the
# input and labels could be made therein also, having make_labels() and pad_sequences() as
# private methods of the class.
#
class LMDataset(Dataset):
    def __init__(self, _encodings, _labels, _nlabels):
        
        assert(len(_encodings) == len(_labels))
        
        self.nsamples = len(_labels)
        
        self.encodings = _encodings # list[list[int]]: contains the padded list of token ids for each sample
        self.labels = _labels # list[list[int]]: contains the label sequence for each sample
        self.nlabels = _nlabels # int: number of labels in the dataset
        
    def __getitem__(self, idx):
        '''
        Returns a dictionary containing the label and padded token ids for a sample
        '''

        return {'ids': torch.tensor(self.encodings[idx]), 'label': torch.tensor(self.labels[idx])}

    def __len__(self):
        return self.nsamples


ds = dict()
MAXLEN = 50
nlabels = len(vocab)

for x in (['train', 'valid', 'test']):
    inputs = pad_sequences(fold[x], MAXLEN)
    labels = make_labels(inputs)

    ds[x] = LMDataset(inputs, labels, nlabels)

## Create the model

### Toy examples of basic operations with embedding and lstm layers in pyTorch

These toy examples on a non-trained models are provided to get a better understanding of how to manipulate the different layers that will constitute the actual model. In particular, we show you how to write a sequence generation loop. 

In [ ]:
#
# Understanding the model forward propagation on a toy example before digging into the actual code
#

BATCHSIZE = 8 # make small batches for illustration purposes

loader = DataLoader(ds['train'], batch_size=BATCHSIZE, shuffle=True)
print('Number of samples:', len(ds['train']))
print(f'Number of training batches:', len(loader))

#
# Model operations
#
batch = next(iter(loader)) # take one batch to play with
vocsize = len(vocab)
dim = 200

embedder = torch.nn.Embedding(vocsize, dim, padding_idx = 0) # embedding layer
rnn = torch.nn.LSTM(dim, dim, batch_first=True) # LSTM recurrence
mlp = torch.nn.Linear(dim, vocsize) # feed forward
softmax = torch.nn.LogSoftmax(dim=-1)

inputs = batch['ids']
print('input shape:', inputs.shape)

embeddings = embedder(batch['ids'])
print(embeddings.shape)

h0 = torch.zeros(1, BATCHSIZE, dim)
c0 = torch.zeros(1, BATCHSIZE, dim)

outputs, _ = rnn(embeddings, (h0, c0)) # lstm(embeddings, hidden0)
print('outputs shape', outputs.shape)

logits = mlp(outputs)
print('logits shape', logits.shape)

probas = softmax(logits)
print('probas shape', probas.shape)


#
# Loss computation
#
# Note the ignore_index argument of the loss function that tells not to compute the
# loss and backpropagate gradient from the points that have that label index. In 
# other words, we don't want to account for the padding in training.
#
# Negative log-likelihood and cross-entropy are alike if the model output consists
# of a probability distribution (after softmax). If the output are logits (before
# softmax then), CrossEntropyLoss() must be used as it includes the normalization.
#
loss_fn = torch.nn.NLLLoss(ignore_index=vocab['PAD'])
# loss_fn = torch.nn.CrossEntropyLoss(ignore_index=vocab['PAD'])
labels = batch['label']
print('labels shape', labels.shape)

# have to permute because of loss function implementation
# see https://discuss.pytorch.org/t/loss-function-format-for-sequence-ner-pos-tagging/57548
loss = loss_fn(probas.permute(0,2,1), labels) 

In [ ]:
#
# Generation loop with such a model
#

#
# random initialization of the reccurence state h (and cell state c since
# LSTMs also have cell states in addition to recurrence state)
#
h = torch.randn(1, dim)
c = torch.randn(1, dim)

#
# initialize utterance to start of sentence token
#
ids = [vocab['BOS']]

#
# iterate up to a maximum length (here 20)
#
for i in range(10):
    inputs = torch.tensor([ids[-1]]) # input to single recurrence step is the last token
    embeddings = embedder(inputs) # get embedding of single/last token    
    outputs, (h, c) = rnn(embeddings, (h, c)) # run one step of the LSTM recurrence
    probas = softmax(mlp(outputs)) # get the pdf over the vocabulary
    
    next_id = torch.argmax(probas) # get best guess -- to get random guess, use torch.multinomial()
    ids.append(next_id)
    
    if next_id == vocab['EOS']: 
        break
    
    print(decode_utterance(ids))
    
    
    
    


### Define and train the actual model

In [ ]:
#
# Define the model as a torch.nn.Module
#

class LMRNN(torch.nn.Module):

    def __init__(self, vocsize, nclasses, embed_dim = 200, dropout = None):
        super(LMRNN, self).__init__()

        self.nclasses = vocsize
        self.vocabulary_size = vocsize
        self.embed_dim = embed_dim
        
        self.embedding = torch.nn.Embedding(vocsize, embed_dim, padding_idx = 0)
        self.rnn = torch.nn.LSTM(embed_dim, embed_dim, batch_first=True)
        self.embedding = torch.nn.Embedding(vocsize, embed_dim, padding_idx = 0)
        if dropout != None:
            self.dropout = torch.nn.Dropout(dropout)
            self.rnn = torch.nn.LSTM(embed_dim, embed_dim, dropout = dropout, batch_first=True)
        else:
            self.dropout = None
            self.rnn = torch.nn.LSTM(embed_dim, embed_dim, batch_first=True) 
        self.linear = torch.nn.Linear(embed_dim, nclasses)
        # self.softmax = torch.nn.LogSoftmax(dim=1)
  
    def forward(self, **kwargs):
        x = self.embedding(kwargs['ids']) # embed input ids -- batch_size * maxlen * embed_dim
        if self.dropout != None: # dropout embeddings
            x = self.dropout(x)        
        x, _ =  self.rnn(x) # run through recurrent cells -- batch_size * maxlen * embed_dim
        x = self.linear(x) # project into posterior probabilities -- batch_size * maxlen * nclasses
        
        return x.permute(0, 2, 1) # because loss function wants batch_size * nclasses * maxlen
    
          
lm = LMRNN(len(vocab), nclasses = ds['train'].nlabels, embed_dim = 200)


In [ ]:
#
# Quick adaptation of the train_step() and eval_step() functions from previous lecture. Only
# the latter requires changes because (i) label prediction accuracy makes little sense for
# language models and (ii) we're dealing with sequences of predictions and not a prediction
# for document.
#
# We will limit ourselves to the negative log-likelihood (the loss function used here) but 
# the real metric that should be monitored is known as perplexity (should implement that one
# day).
#

def train_step(_model, _loader, _loss, _optim, device="cpu", report=0):
    '''
    Generic training step.

    Assumes loader returns batches where the labels are accessed with the 'label' keyword.
    All other keywords are passed as **kwargs to the model.
    
    If report is set to a number, reports stats on training every 'report' batches.

    :return: total_loss accumulated throughout the epoch
    '''

    _model.train(True)
    total_loss = 0.
    running_loss = 0.

    for i, batch in enumerate(_loader):
        _optim.zero_grad()

        labels = batch['label'].to(device)

        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        outputs = _model(**inputs)

        loss = _loss(outputs, labels)
        total_loss += loss.item()
        running_loss += loss.item()

        loss.backward()
        _optim.step()

        if report != 0 and i % report == report - 1:
            print('  batch {} avg. loss per batch={:.4f}'.format(i + 1, running_loss / report))
            running_loss = 0.

    _model.train(False)

    return total_loss

def eval_step(_model, _loader, device='cpu', loss_fn=None):
    '''
    Evaluate the model's performance on data within loader.
    
    :return: 
    total_loss accumulated throughout the batches
    '''
    
    _model.eval()  # disable training mode

    total_loss = 0.

    for batch in _loader:
        labels = batch['label'].to(device)

        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}

        with torch.no_grad():
            outputs = _model(**inputs)

        if loss_fn != None:
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
    
    return total_loss

In [ ]:
#
# Get ready with the necessary equipment for training: parameters, optimizer, loss, etc.
#

lr = 5e-4
nepochs = 20
report_freq = 50
batch_size = 32

# check what device we can work on
if torch.backends.mps.is_built(): # MPS GPU library for MacOS -- requires metal to be installed
    device = "mps"
    torch.mps.empty_cache()
elif torch.cuda.is_available(): # CUDA GPU acceleration available
    device = torch.device('cuda')
else:
    device = "cpu"
print(f'Running on {device} device')

# define optimizer and loss
optimizer = torch.optim.AdamW(lm.parameters(), lr=lr)
celoss = torch.nn.CrossEntropyLoss(ignore_index=vocab['PAD'])

# create batches within loaders
loader = dict()
loader['train'] = DataLoader(ds['train'], batch_size=batch_size, shuffle=True) # set to False for debugging purposes
loader['valid'] = DataLoader(ds['valid'], batch_size=batch_size)
loader['test'] = DataLoader(ds['test'], batch_size=batch_size)

In [ ]:
#
# And finally run the training loop
#


lm.to(device)

for epoch in range(nepochs):
    print(f'epoch: {epoch}')
    
    total_loss = train_step(lm, loader['train'], celoss, optimizer, device=device, report=report_freq)
    
    val_loss = eval_step(lm, loader['valid'], device=device, loss_fn=celoss)

    print('  **train** avg_loss={:.4f}'.format(total_loss / len(loader['train'])))
    print('  **valid** avg_loss={:.4f}'.format(val_loss / len(loader['valid'])))


In [ ]:
#
# This cell teaches you how to save/load a model. Because training takes a bit of time with 
# recurrent neural networks, you'd better save your model once trained so that you can load
# it back at any point and play with generation.
# 
# 
fn = 'data.NOSAVE/rnnlm-imdb-pos.pt'

# torch.save(lm.state_dict(), fn)

#
# The following lines assume the instance lm of the LMRNN() class has been created and simply
# reload the weights.
#
lm.load_state_dict(torch.load(fn, weights_only=True))
lm.eval()

## Play with the model to generate texts

Now that you have a language model fully trained, you should be able to generate texts and/or compute
the probability of a given input. At the very least, you should write a function that randomly generates
a sentence given the model. If time permits or if curiosity keeps you awake, you could extend this 
generation function to be able to complete a given prompt: i.e., run tghe prompt through the RNN to 
get the RNN states before startig the generation loop.

Keep in mind that the model as defined above outputs logits and not sofmax-normalized probabilities.

Note that in practice, these functions are usually defined as methods within the model class.

In [ ]:

def random_generate(_model, maxlen: int = 20) -> list[int]:
    '''
    Random generation of a text given the model.
    '''
    
    return []
    

    
def prompt_generate(_model, _promt: list[int], maxlen: int = 20) -> list[int]:
    '''
    Complete prompt with the most likely completion for the given the model.
    '''
    
    
    return []
